<a href="https://colab.research.google.com/github/mdi-group/mace-field-tutorial/blob/main/MACE_Field_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MACE Field Tutorial




In this tutorial, we dive into the alterations we have made to the MACE model to incorporate an external perturbing electric field into the MACE architecture, and how to use it to derive derivative properties such as the macroscopic polarisation, Born Effective Charges (BECS) and polarisability.

To learn how the base MACE code works, we highly recommend you look at the [MACE theory tutorial](https://colab.research.google.com/drive/1AlfjQETV_jZ0JQnV5M3FGwAM2SGCl2aU) developed by Will Baldwin and Ilyes Batatia.

## Installs

In [1]:
!rm -rf /usr/local/lib/python3.11/site-packages/mace
!git clone https://github.com/mdi-group/mace-field-tutorial.git
!git clone https://github.com/mdi-group/mace-field.git
%cd mace-field
!git switch field
!pip install .
!pip install --force-reinstall numpy==2.0
%cd ../mace-field-tutorial

Cloning into 'mace-field-tutorial'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 47 (delta 19), reused 8 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (47/47), 2.27 MiB | 5.90 MiB/s, done.
Resolving deltas: 100% (19/19), done.
Cloning into 'mace-field'...
remote: Enumerating objects: 6184, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 6184 (delta 39), reused 30 (delta 28), pack-reused 6124 (from 3)
Receiving objects: 100% (6184/6184), 123.65 MiB | 11.04 MiB/s, done.
Resolving deltas: 100% (4636/4636), done.
Updating files: 100% (98/98), done.
/content/mace-field
Branch 'field' set up to track remote branch 'field' from 'origin'.
Switched to a new branch 'field'
Processing /content/mace-field
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyp

In [2]:
import numpy as np
import torch

# Add this line before importing e3nn
torch.serialization.add_safe_globals([slice])  # Allow the 'slice'

import torch.nn.functional
from e3nn import o3
from matplotlib import pyplot as plt

from ase.io import read, write

from mace import data, modules, tools
from mace.tools import torch_geometric
torch.set_default_dtype(torch.float64)

import warnings
warnings.filterwarnings("ignore")

from mace.cli.run_train import main as mace_run_train_main
import sys
import logging

In [ ]:
from typing import Any, Dict, Optional
from mace.tools.scatter import scatter_sum
from mace.modules.blocks import (
    LinearReadoutBlock,
    ScaleShiftBlock,
)
from mace.modules.utils import (
    get_edge_vectors_and_lengths,
    get_symmetric_displacement,
)
from mace.modules.models import (
    MACE,
    ScaleShiftFieldMACE
)

## Write Config for the Model

In [4]:
ferroelectrics_train = read("data/ferroelectric/ferroelectric_train_2040.xyz", ":")[:1]
ferroelectrics_valid = read("data/ferroelectric/ferroelectric_valid.xyz", ":")[:1]
ferroelectrics_test = read("data/ferroelectric/ferroelectric_test.xyz", ":")[:1]
write("data/ferroelectric/ferroelectric_train_1.xyz", ferroelectrics_train)
write("data/ferroelectric/ferroelectric_valid_1.xyz", ferroelectrics_valid)
write("data/ferroelectric/ferroelectric_test_1.xyz", ferroelectrics_test)

/content/mace-field-tutorial


In [ ]:
%%writefile train_polarisation.yml

name: "ferroelectrics"
train_file: "data/ferroelectric/ferroelectric_train_1.xyz"
test_file: "data/ferroelectric/ferroelectric_test_1.xyz"
valid_file: "data/ferroelectric/ferroelectric_valid_1.xyz"
E0s: "average"
loss: "universal_field"
energy_weight: 0.0
forces_weight: 0.0
stress_weight: 0.0
bec_weight: 0.0
polarisability_weight: 1e7
polarisation_weight: 0.0
compute_field: True
eval_interval: 1
error_table: "PerAtomRMSEstressvirialsfield"
model: "ScaleShiftFieldMACE"
interaction_first: "RealAgnosticResidualInteractionBlock"
interaction: "RealAgnosticResidualInteractionBlock"
num_interactions: 2
correlation: 3
r_max: 6.0
max_L: 1
max_ell: 3
num_channels: 128
num_radial_basis: 10
MLP_irreps: "16x0e"
num_workers: 1
lr: 0.05
weight_decay: 1e-8
batch_size: 1
valid_batch_size: 1
max_num_epochs: 50
device: cuda
seed: 1

## Training a MACE Field model

In [ ]:
def train_mace_field(config_file_path):
    logging.getLogger().handlers.clear()
    sys.argv = ["program", "--config", config_file_path]
    mace_run_train_main()

In [ ]:
train_mace_field("train_polarisation.yml")

2025-04-16 08:29:59.002 INFO: ===========VERIFYING SETTINGS===========
2025-04-16 08:29:59.003 INFO: MACE version: 0.3.10
2025-04-16 08:29:59.092 INFO: CUDA version: 12.4, CUDA device: 0
2025-04-16 08:29:59.287 INFO: ===========LOADING INPUT DATA===========
2025-04-16 08:29:59.288 INFO: Using heads: ['default']
2025-04-16 08:29:59.291 INFO: =============    Processing head default     ===========
2025-04-16 08:29:59.298 INFO: Training set [1 configs, 1 energy, 252 forces] loaded from 'data/ferroelectric/ferroelectric_train_1.xyz'
2025-04-16 08:29:59.303 INFO: Validation set [1 configs, 1 energy, 132 forces] loaded from 'data/ferroelectric/ferroelectric_valid_1.xyz'
2025-04-16 08:29:59.308 INFO: Test set (1 configs) loaded from 'data/ferroelectric/ferroelectric_test_1.xyz':
2025-04-16 08:29:59.309 INFO: Default_default: 1 configs, 1 energy, 120 forces
2025-04-16 08:29:59.311 INFO: Total number of configurations: train=1, valid=1, tests=[Default_default: 1],
2025-04-16 08:29:59.312 INFO:

## Understanding the MACE Field Architecture

### Theory

Our approach inherits most of the original MACE architecture. The primary alteration is in the readout blocks where we include an additional energy term, $-\Omega\ \mathbf{P} \cdot \mathcal{E}$, where $\Omega$ is the unit-cell volume, $\mathbf{P}$ is the macroscopic polarisation, and $\mathcal{E}$ is an external electric field. This is all in analogy to the electric enthalpy functional from Density Functional Perturbation Theory (DFPT). Please see the [VASP wiki](https://www.vasp.at/wiki/index.php/Berry_phases_and_finite_electric_fields) for a good introduction to Berry Phases and finite electric fields in DFPT.

The molecular dipole moment is the sum of the ionic contributions, with bare ionic charge $-e Z_\alpha$ and position $\mathbf{R}_\alpha$, and an electronic contribution from the first moment of the electronic charge density $\rho(\mathbf{r})$:


\begin{equation}
\begin{aligned}
    \mathbf{p} &= \mathbf{p}_{\text{ion}} + \mathbf{p}_{\text{el}}\\
    \mathbf{p} &= -e \sum_{\alpha} Z_\alpha \mathbf{R}_\alpha + \int d\mathbf{r}\ \mathbf{r}\ \rho(\mathbf{r})
\end{aligned}
\end{equation}


where the polarisation density is then just this dipole moment divided by the total volume, $\mathbf{P} = \mathbf{p} / V$.


We see that the molecular dipole $\mathbf{p}_{\text{ion}, \alpha}$ has a contribution per ion. Just as we decompose the total energy of the system into contributions per ion or per "node", suppose we decompose the electronic dipole into contributions per node, $\mathbf{p}_{\text{el}, \alpha}$.

Due to the Modern Theory of Polarisation, quantum polarisation is multivalued / "ill-defined" for infinite periodic systems. The electronic dipole cannot in principle be decomposed this way, but just as is the case for the total energy, we will do it anyway. To compensate, the loss term for the polarisation will only be defined modulo the polarisation quantum to account for this multivalueness.

### Alterations to MACE

Each layer $1 \leq t \leq T$ of MACE contributes to the final energy readout $E_\alpha$ for node $\alpha$:

\begin{equation}
    E_\alpha = E_\alpha^{(0)} + E_\alpha^{(1)} + \dots + E_\alpha^{(T)}
\end{equation}

In a $T$-layer MACE, the readout is altered to include an additional perturbing term of a $K$-dimensional "total atomic dipole" feature $\mathbf{p}_{\alpha, k}$ for each node $\alpha$, dot-producted with the external electric field $\mathcal{E}$:

\begin{equation}
  E_\alpha(t) = \mathcal{R}_t\left(\mathbf{h}_i^{(t)}\right) =
    \begin{cases}
    \begin{aligned}
        &\sum_{\tilde{k}} W_{\text{readout}, \tilde{k}}^{(t)} \left[ h_{\alpha,\tilde{k} 0 0}^{e, (t)} - \mathbf{p}_{\alpha, \tilde{k}}^{(t)} \cdot \mathbf{\mathcal{E}} \right] \qquad\ \text{if}\ t<T, \\
        &\text{MLP}_{\text{readout}}^{(t)}\left( \left\{ h_{\alpha,k 0 0}^{e, (t)} - \mathbf{p}_{\alpha, k}^{(t)} \cdot \mathbf{\mathcal{E}} \right\}_{k} \right) \quad \text{if}\ t = T.
    \end{aligned}
    \end{cases}
\end{equation}

Where in the final layer the electric field enters the nonlinear MLP.

After the higher body-order `node_feats` are produced from the standard `product` blocks in the MACE model, we linearly map them to two scalar features and one vector feature which we may relate to the local node energy "$e$", charge "$q$" and electronic dipole moment:

\begin{equation}
    \left[ h_{\alpha, k 0 0}^{e, (t)},\ h_{\alpha, k 0 0}^{q, (t)},\ h_{\alpha, k 1 m}^{(t)} \right] = \sum_{l \tilde{m}} W^{l \tilde{m}}_{0 0, 1 m} h^{(t)}_{\alpha,kl\tilde{m}}.
\end{equation}

Note that these are not complete readouts yet as we have not yet mixed the k channels.

To extract these features, we need to initialise in `__init__()` a new readout:

```
field_irreps = o3.Irreps("0e") + o3.Irreps.spherical_harmonics(1)
field_irreps_out = o3.Irreps(f"{num_channels * field_irreps}").sort()[0].simplify()

self.field_readout = LinearReadoutBlock(hidden_irreps, field_irreps_out)
```

which gives us two $l=0$ scalars and a $l=1$ vector *without* mixing the channels.

After we obtain `node_feats` from the `product` block, we obtain these additional features from:

```
node_out = self.field_readout(node_feats, node_heads).reshape(node_feats.shape[0], -1, 5)
node_energy_feats, node_charge_feats, node_electronic_dipole_feats = node_out[:,:,0],  node_out[:,:,1], -node_out[:,:,2:]
```

We then define a total dipole moment feature as:

\begin{equation}
    \mathbf{p}^{(t)}_{\alpha,k} = h^{q,(t)}_{\alpha,k 0 0} \mathbf{R}_{\alpha} - \mathbf{h}^{(t)}_{\alpha,k 1},
\end{equation}

where the first term represents the ionic dipole contribution.

In the code we have:

```
node_atomic_dipole_feats = torch.einsum('ij,ik->ijk', node_charge_feats, data["positions"])
node_dipole_feats = node_atomic_dipole_feats + node_electronic_dipole_feats           
```

This "total dipole" acts as an atomic decomposition of the total macroscopic polarisation which, dot product with the external electric field, contributes to the total energy:

\begin{equation}
h_{\alpha,\tilde{k} 0 0}^{e, (t)} - \mathbf{p}_{\alpha, \tilde{k}}^{(t)} \cdot \mathbf{\mathcal{E}},
\end{equation}

which we then feed into the final energy `readout` block which mixes the $k$ channels:

```
node_energies = node_energy_feats - torch.einsum('ijk,k->ij', node_dipole_feats, data["electric_field"])
node_energies = readout(node_energies, node_heads)[num_atoms_arange, node_heads]        
```

Since we need the node features, `node_feats`, to always have a $l=1$ piece, we need to alter the final `interaction` and `product` blocks of MACE which originally only preserve the $l=0$ piece in the last layer $T$.

Therefore, in `__init__()` in our new model `ScaleShiftFieldMACE` we also include:
```
self.interactions[-1].skip_tp = o3.FullyConnectedTensorProduct(
    hidden_irreps,
    self.interactions[-1].node_attrs_irreps,
    hidden_irreps,
    self.interactions[-1].cueq_config,
)
```
and
```
self.products[-1] = self.products[-2]
```

Finally, as our new `field_readout` is an intermediate step between the `product` block and the energy `readout`, we need to alter the `irreps_in` of the energy `readout` blocks:
```
for i in range(len(self.readouts)-1):
    self.readouts[i].linear = o3.Linear(f"{num_channels}x0e", f"{len(self.heads)}x0e")

self.readouts[-1].linear_1 = o3.Linear(f"{num_channels}x0e", f"{len(self.heads) * kwargs['MLP_irreps']}")   
```

### Deriving Polarisation, BECs and Polarisability as derivates of energy

The predicted energy $E$ depends upon the atomic positions $\mathbf{R}_{\alpha}$ and the external electric field $\mathbf{\mathcal{E}}$ to arbitrary order,

\begin{equation}
    E(\{\mathbf{R}_\alpha\}, \mathbf{\mathcal{E}}) = E^{(0)}(\mathbf{\mathcal{E}}) + \sum_{\alpha=1}^N E_{\alpha}^{(0)}\left( \mathbf{R}_{\alpha};\  \mathbf{\mathcal{E}} \right) + \sum_{1 \leq \alpha \leq \beta \leq N}^N E_{\alpha, \beta}^{(1)}\left( \mathbf{R}_{\alpha}, \mathbf{R}_{\beta};\ \mathbf{\mathcal{E}} \right) + \dots + \mathcal{F}\left[m^{(T)}\left(\mathbf{R}_{\alpha_1}, \dots, \mathbf{R}_{\alpha_N};\  \mathbf{\mathcal{E}}\right)\right],
\end{equation}  

where $\mathcal{F}$ is a general, learnable non-linear term (here evaluated using a MLP) that accounts for excluded higher-order terms.

Therefore, we account for non-linear susceptibilities,

\begin{equation}
    \mathbf{P}(\{\mathbf{R}_\alpha\}, \mathbf{\mathcal{E}}) = -\frac{1}{\Omega} \mathbf{\nabla}_{\mathbf{\mathcal{E}}} E(\{\mathbf{R}_\alpha\}, \mathbf{\mathcal{E}}) = \mathbf{P}_0(\{\mathbf{R}_\alpha\}) + \chi^{(1)}(\{\mathbf{R}_\alpha\}) \cdot  \mathbf{\mathcal{E}} + \mathbf{\mathcal{E}} \cdot \chi^{(2)}(\{\mathbf{R}_\alpha\}) \cdot  \mathbf{\mathcal{E}} + \dots,
\end{equation}

where we see that the polarisability is the linear susceptibility term, $\chi^{(1)} \equiv \alpha$.

The polarisation ($P_i$) is then derived from the derivative of the total energy with respect to the external electric field:

\begin{equation}
  P_i = -\frac{1}{\Omega} \frac{\partial E}{\partial \mathcal{E}_i},
\end{equation}

which we compute using the forward `autograd.grad` function:

```
def compute_polarisation(
    energy: torch.Tensor,
    electric_field: torch.Tensor,
    cell: torch.Tensor,
    training: bool = True,
) -> torch.Tensor:
    
    volume = torch.linalg.det(cell.view(-1, 3, 3)).abs()
    
    polarisation = torch.autograd.grad(
        outputs=energy,  # [n_graphs, ]            
        inputs=electric_field,  # [3, ]
        grad_outputs=torch.ones_like(energy),
        retain_graph=training,  # Make sure the graph is not destroyed during training  
        create_graph=training,  # Create graph for higher derivatives
        allow_unused=True,
    )[0]

    return -polarisation / volume
```


The Born effective charges ($Z^*_{\alpha,ij}$) and the polarisability tensor ($\alpha_{ij}$) may then be derived from derivatives of the polarisation with respect to atom position and the electric field, respectively,

\begin{equation}
    Z^*_{\alpha, ij} = -\frac{1}{e}\frac{\partial^2 E}{\partial \mathcal{E}_i\ \partial R_{\alpha,j}}\bigg\rvert_{\mathbf{\mathcal{E}}=\mathbf{0}}
    = \frac{1}{e} \frac{\partial F_i}{\partial \mathcal{E}_j}\bigg\rvert_{\mathbf{\mathcal{E}}=\mathbf{0}} = \frac{\Omega}{e} \frac{\partial P_i}{\partial R_{\alpha, j}}\bigg\rvert_{\mathbf{\mathcal{E}}=\mathbf{0}},
\end{equation}

\begin{equation}
    \alpha_{ij} = - \frac{1}{\Omega} \frac{\partial^2 E}{\partial \mathcal{E}_i\ \partial \mathcal{E}_j} = \frac{\partial P_i}{\partial \mathcal{E}_j},
\end{equation}

which are computed by looping over the $3$ dimensions of the polarisation vector:

```
def compute_bec(
    polarisation: torch.Tensor,
    positions: torch.Tensor,
    cell: torch.Tensor,
    training: bool = True,
) -> torch.Tensor:
    
    volume = torch.linalg.det(cell.view(-1, 3, 3)).abs()
    
    bec_polar_list = []
    for d in range(3): # Loop over dimensions
        polar_component = polarisation[d]
        gradient = torch.autograd.grad(
            outputs=polar_component, # [n_graphs, 1]
            inputs=positions, # [n_nodes, 3]
            grad_outputs=torch.ones_like(polar_component),
            retain_graph=training,  # Make sure the graph is not destroyed during training
            create_graph=training,  # Create graph for higher derivatives
            allow_unused=True,
        )[0]
        bec_polar_list.append(gradient) # [n_nodes, 3]
        
    bec = torch.stack(bec_polar_list, dim=1) # [n_nodes, 3, 3]

    return bec * volume
```

```
def compute_polarisability(
    polarisation: torch.Tensor,
    electric_field: torch.Tensor,
    training: bool = True,
) -> torch.Tensor:
    
    # Second derivatives (BEC and polarisability) computed for each polarisation component.   
    polarisability_list = []
    for d in range(3):
        polar_component = polarisation[d]
        grad_field = torch.autograd.grad(
            outputs=polar_component, # [n_graphs, 1]
            inputs=electric_field, # [3, ]
            grad_outputs=torch.ones_like(polar_component),
            retain_graph=training,  # Make sure the graph is not destroyed during training
            create_graph=training,  # Create graph for higher derivatives
            allow_unused=True,
        )[0]
        polarisability_list.append(grad_field) # [n_graphs, 3]
        
    polarisability = torch.stack(polarisability_list, dim=1)  # [n_graphs, 3, 3]

    return polarisability
```

### Additional Loss Term

In addition to the original loss function for the energy, forces and stress, we include three additional terms to account for the polarisation, becs and polarisability:

\begin{equation}
\begin{aligned}
    \Delta \mathcal{L} &= \frac{\lambda_P}{3 B} \sum_{b=1}^{B} \sum_{i=1}^3 \left(\left[-\frac{1}{\Omega}\frac{\partial E_b^{(\text{pred})}}{\partial \mathcal{E}_i} - P^{(\text{ref})}_{b,i}\right] \text{mod}\ \Delta P_{b,i} \right)^2 + \frac{\lambda_Z}{9 B N} \sum_{\alpha=1}^{B \cdot N} \sum_{i=1}^3 \sum_{j=1}^3 \left(-\frac{1}{e}\frac{\partial^2 E^{(\text{pred})}}{\partial \mathcal{E}_i\ \partial R_{\alpha,j}} - Z^{* (\text{ref})}_{\alpha, ij} \right)^2 \\
    &+ \frac{\lambda_\alpha}{9 B} \sum_{b=1}^{B} \sum_{i=1}^3 \sum_{j=1}^3 \left(-\frac{1}{\Omega}\frac{\partial^2 E_b^{(\text{pred})}}{\partial \mathcal{E}_i\ \partial \mathcal{E}_j} - \alpha^{(\text{ref})}_{b,ij} \right)^2 .
\end{aligned}
\end{equation}

Here $B$ is the size of the batch and $b$ the batch index. $N$ is the number of atoms in the graph. The $\lambda$s are the weights which will be automatically set to zero if the relevant training data is absent. This means a mixed dataset can be used for training.

Here we write the MSE loss, but other losses can be used. In the main code we actually use the `torch.nn.HuberLoss` Huber loss with $\delta = 0.1$.

The modulo $\text{mod}\ \Delta P_{b,i}$ term tells the model that the polarisation may be multivalued due to the Berry Phase, where the $ \Delta \mathbf{P} $ term is the polarisation quantum:

\begin{equation}
   \Delta \mathbf{P} = \frac{e \mathbf{R}}{\Omega},
\end{equation}

where $\mathbf{R}$ is the lattice vector.

To implement this, we `torch.repeat` the reference and predicted polarisation vectors into a $3\times3$ matrices and compare their difference modulo the $3\times3$ `Cell` object from the `ASE` `Atom` type , weighted by the volume:

```
# Calculate the polarisation quantum
cell = ref["cell"].view(-1,3,3)
polarisation_quantum = cell / torch.linalg.det(cell).abs()

# modulo ignore zero components to leave pol unfolded and avoid divide by 0
polarisation_quantum[polarisation_quantum == 0] = max(torch.cat((ref["polarisation"], pred["polarisation"]))) + 1.0

# Expand polarisation to lattice (3x3 matrix) that is modulo the polarisation quantum. Any nan (due to divide by 0), set to zero.
ref_polarisation = ref["polarisation"].repeat(3,1).view(-1,3,3)
pred_polarisation = pred["polarisation"].repeat(3,1).view(-1,3,3)

polarisation_loss = self.huber_loss((ref_polarisation - pred_polarisation).fmod(polarisation_quantum).nan_to_num(nan=0) / num_atoms, torch.zeros_like(ref_polarisation))
        
```

The [`Cell` object](https://wiki.fysik.dtu.dk/ase/ase/cell.html) is simply the three lattice vectors forming a parallelepiped.

The weird manipulation of the polarisation quantum here is just to avoid dividing by zero in the modulo and to ignore any null directions where the lattice vector has zero entries. The difference modulo the `Cell` object is the compared to a $3\times3$ null matrix.